# Una breve introduccion a lo que queremos del curso

Este notebook introduce un ejemplo de lo que queremos que sepan hacer hacia el final del curso. Conceptualmente, el procedimiento es:

*   Explorar los datos
*   Plantear el problema a resolver
*   Preprocesar los datos a un formato adecuado
*   Elegir algoritmos
*   Fittear y validar
*   Decidir el algoritmo final, y testear




Antes que nada, importamos algunos paquetes

In [ ]:
import numpy as np
import os
import sys
import tarfile
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
# to make this notebook's output stable across runs
np.random.seed(42)
import pandas as pd
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


# Los datos

Vamos a utilizar el dataset de California. Una buena practica es, si los datos lo permiten, separar un conjunto de test que voy a utilizar solamente al final de todo, para evaluar todo lo que hice.

## Traemos los datos

In [ ]:
HOUSING_PATH = "datasets"
def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [ ]:
if 'google.colab' in sys.modules:
        
    import tarfile

    DOWNLOAD_ROOT = "https://github.com/ageron/handson-ml2/raw/master/"
    HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

    !mkdir -p ./datasets/housing

    def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
        os.makedirs(housing_path, exist_ok=True)
        tgz_path = os.path.join(housing_path, "housing.tgz")
        #urllib.request.urlretrieve(housing_url, tgz_path)
        !wget {HOUSING_URL} -P {housing_path}
        housing_tgz = tarfile.open(tgz_path)
        housing_tgz.extractall(path=housing_path)
        housing_tgz.close()

    # Corramos la función
    fetch_housing_data()

else: 
    print("Not running on Google Colab. This cell is did not do anything.")

## Preprocesamos un poco

In [ ]:
housing_pre = load_housing_data()

### Train/test splitting

Como bien discutimos, es menester separar los datos en dos conjuntos: el de entrenamiento y el de testeo. Este ultimo debe utilizarse al FINAL del proyecto para garantizar una prediccion no sesgada de la performance del modelo final.

La opcion mas sencilla es utilizar `train_test_split`, donde especificamos el porcentaje de datos que separamos para testear.

In [ ]:
train_df, test_df = train_test_split(housing_pre, test_size=0.2, random_state=42)

Un problema que puede aparecer es que este splitting no sea representativo en algunos features. Por ejemplo, `median_income`. Definiendo una variable auxiliar `income_cat` con los valores binneados de `median_income` podemos estudiar esto

In [ ]:
housing_pre["income_cat"] = pd.cut(housing_pre["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

plt.hist([housing_pre[housing_pre["income_cat"] == cat].median_income for cat in range(1,6)], 
         label =  list(range(1,6)), 
         bins=50,
         stacked=True)
plt.legend()
plt.ylabel('# of districts')
plt.xlabel('Median House Income')
plt.show()
plt.hist(housing_pre['income_cat'], density=True)
plt.xticks([1,2,3,4,5])
plt.ylabel('# of districts')
plt.xlabel('Median House Income Category')
plt.show()

housing_pre['income_cat'].value_counts() / len(housing_pre)

Veamos que pasa con `income_cat` con el `train_test_split` por defecto

In [ ]:
train_df, test_df = train_test_split(housing_pre, test_size=0.2, random_state=42)
plt.hist(train_df['income_cat'], density=True)
plt.xticks([1,2,3,4,5])
plt.ylabel('# of districts')
plt.xlabel('Median House Income Category')
plt.title('Train')
test_df['income_cat'].value_counts() / len(test_df)

Otra opcion es obligar a respetar proporciones utilizando `StratifiedShuffleSplit`

In [ ]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=445543)
for train_index, test_index in split.split(housing_pre, housing_pre["income_cat"]):
    california_housing_train = housing_pre.loc[train_index]
    california_housing_test = housing_pre.loc[test_index]

#for set_ in (california_housing_train, california_housing_test):
#    set_.drop("income_cat", axis=1, inplace=True)

plt.hist(california_housing_train['income_cat'], density=True)
plt.xticks([1,2,3,4,5])
plt.ylabel('# of districts')
plt.xlabel('Median House Income Category')
plt.title('Train')
california_housing_test['income_cat'].value_counts() / len(california_housing_test)    

Para comparar ambos metodos, podemos hacer

In [ ]:
comparison_df = pd.concat([housing_pre['income_cat'].value_counts() / len(housing_pre), test_df['income_cat'].value_counts() / len(test_df),california_housing_test['income_cat'].value_counts() / len(california_housing_test)], axis=1)
comparison_df.columns = ['original', 'random_split', 'stratified_split']
comparison_df

La diferencia no es _enorme_ pero puede ser importante, especialmente para datasets chicos/

Una vez que decidi, puedo sacarme de encima esta categoria auxiliar

In [ ]:
for set_ in (california_housing_train, california_housing_test):
    set_.drop("income_cat", axis=1, inplace=True)


In [ ]:
california_housing_train.info()

In [ ]:
california_housing_test.info()

## Exploremos los datos (nuevamente)

In [ ]:
housing=california_housing_train.copy()

In [ ]:
housing.hist(bins=50, figsize=(20,15))

In [ ]:
attributes = ["median_house_value", "median_income", "total_rooms",
              "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12, 8))

In [ ]:
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)


### Valores raros

Como vimos en la clase pasada, hay features con valores raros

In [ ]:
columns = housing.columns.to_list()

N_col = 4
N = len(columns)
N_rows = int(np.ceil(N/N_col))

fig, ax = plt.subplots(N_rows,N_col, figsize=(5*N_col,5*N_rows))

for i in range(N_rows):
    for j in range(N_col):
        ax[i,j].hist(housing[columns[i*N_rows+j]], bins=50)
        ax[i,j].set_title(columns[i*N_rows+j])

Vemos que existe una saturacion en algunos features

In [ ]:
problematic_columns = ['median_house_value', 'housing_median_age', 'median_income']
max_values=[]
for col in problematic_columns:
    max_value = housing[col].max()
    print(f"{col}: {sum(housing[col] == max_value)} districts with {col} = {max_value} ({round(sum(housing[col] == max_value)/len(housing)*100,2)}%).")
    max_values.append(max_value)

Cuando pasa esto, tenemos que decir que hacemos. Una opcion es descartarlos y poner una cota a partir de la cual no confiamos en el modelo

In [ ]:
housing_clean = housing.copy()
for col, max_value in zip(problematic_columns, max_values):
    housing_clean = housing_clean[housing_clean[col] != max_value]

In [ ]:
columns = housing_clean.columns.to_list()

N_col = 4
N = len(columns)
N_rows = int(np.ceil(N/N_col))

fig, ax = plt.subplots(N_rows,N_col, figsize=(5*N_col,5*N_rows))

for i in range(N_rows):
    for j in range(N_col):
        ax[i,j].hist(housing_clean[columns[i*N_rows+j]], bins=50)
        ax[i,j].set_title(columns[i*N_rows+j])


Pero ojo! si hacemos esto en entrenamiento tambien tenemos que hacerlo con el conjunto de testeo. Pero ahora utilizamos los max_values ya aprendidos, no volvemos a aprenderlos.

In [ ]:
housing_test=california_housing_test.copy()
housing_test_clean = housing_test.copy()
for col, max_value in zip(problematic_columns, max_values):
    housing_test_clean = housing_test_clean[housing_test_clean[col] != max_value]

### Features faltantes

Si nos fijamos bien en los datos, vemos algo molesto

In [ ]:
housing_clean.info()

En efecto, `total_bedrooms` esta incompleto! Cuando pasa esto, tenemos _a grosso modo_ tres opciones

1. Excluir los features incompletos del analisis (esto reduce la cantidad de columnas)
2. Excluir las observaciones o datos donde faltan features del analisis (esto reduce la cantidad de filas).
3. Rellenar los valores faltantes con datos sinteticos utilizando algun criterio.

La eleccion optima depende, como siempre, de los datos y del problema. Nosotros vamos a usar la opcion 3 y llenamos `total_bedrooms` con la mediana.

Podemos hacerlo a mano o aprovechar `sklearn` y utilizar `SimpleImputer`

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')

train_imputer = imputer.fit_transform(housing_clean.drop(['ocean_proximity'], axis=1))

Veamos que funciona

In [ ]:
train_imputer[np.where(housing_clean['total_bedrooms'].isnull()>0),4]

Nuevamente, si lo hacemos en entrenamiento tenemos que hacerlo en testeo. Esto se hace utiliznado unicamente `transform`, no volviendo a fittear

In [ ]:
test_imputer = imputer.transform(housing_test_clean.drop(['ocean_proximity'], axis=1))

### Variables categoricas

Las variables categoricas presentan un desafio a la hora de entrenar. Los modelos necesitan numeros reales. Para transformar una variable categorica en un numero real se suelen considerar dos opciones:

*   Asignar un numero a cada categoria. En el caso binario, se suele tomar 0 y 1, para distinguir entre "apagado" y "prendido". Para el caso de $\geq2$ categorias, esto tiene sentido si existe un "orden" que sigue la numericacion.
*   Mapear las categorias a un espacio de menor dimensionalidad (lo que se conoce como "embedding".

Un ejemplo de lo segundo es el `One Hot Encoding`. Si hay K categorias posibles para la variable, se mapea cada medicion a un vector de K dimensiones con 0s en todos lados salvo en el lugar correspondiente a su categoria.

In [ ]:
housing_clean['ocean_proximity'].hist()
housing_clean['ocean_proximity'].value_counts()

In [ ]:
ohe=OneHotEncoder()
housing_cat_ohe=ohe.fit_transform(housing_clean[['ocean_proximity']])

In [ ]:
housing_cat_ohe

In [ ]:
housing_cat_ohe.toarray()

Veamos a que se refiere ese encodeo

In [ ]:
housing_clean[['ocean_proximity']]

### Estandarizacion

Cuando tenemos muchas features continuas, podemos tener problemas de unidades. Para evitar eso y que el algoritmo no asigne importancias espurias, conviene estandarizar. Estandarizar es fijar una estrategia para pasar los valores al intervalo [0,1], [-1,1] o lo que sea.
 
En particular, el `StandardScaler` transforma a x en "cantidad de desviaciones estandar de la media:

$x\rightarrow \frac{x-\mu}{\sigma}$

In [ ]:
scaler = StandardScaler()

train_num_scaled = scaler.fit_transform(housing_clean.drop("ocean_proximity", axis=1))
test_num_scaled = scaler.transform(housing_clean.drop("ocean_proximity", axis=1))

### Agregando features

La ingenieria de datos tambien ayuda! En particular, podemos agregar los siguientes features

In [ ]:
housing_clean["rooms_per_household"] = housing_clean["total_rooms"]/housing_clean["households"]
housing_clean["bedrooms_per_room"] = housing_clean["total_bedrooms"]/housing_clean["total_rooms"]
housing_clean["population_per_household"]=housing_clean["population"]/housing_clean["households"]

In [ ]:
housing_clean.info()

In [ ]:
set(housing_clean["ocean_proximity"].values)

# Definion del problema y target

El objetivo es poder predecir la mediana del precio de un distrito por sus caracteristicas. Es un problema de regresion univariada supervisada donde mi target es "median_house_value" y mis features son todas las otras categorias.

Dado que es un problema de regresion, voy a usar una de las metricas mas comunes. El root mean squared error. Si mi target es $\vec{t}=(t_1,t_2,...,t_N)^{T}$ y mis predicciones son $\vec{y}=(y_1,y_2,...,y_N)^{T}$, entonces

$\text{RMSE}(\vec{t},\vec{y})=\sqrt{\frac{1}{N}\sum_{n=1}^{N}(t_n-y_n)^{2}}$

La idea del RMSE es dar un error esperado a la prediccion

In [ ]:
from sklearn.metrics import mean_squared_error

def rmse(y,t):
  return np.sqrt(mean_squared_error(y,t))

In [ ]:
t_test_rmse=[0.1,0.3,-0.1]
y_test_rmse=[0.05,0.35,-0.05]
print(rmse(t_test_rmse,y_test_rmse))

# Preprocesado de datos con pipeline


Voy a separar el target y escalear las variables numericas y re-expresar las categoricas. Combino todo en un pipeline.

In [ ]:
housing_labels = housing_clean["median_house_value"].copy()
# label_scaler=StandardScaler()
# housing_labels_scaled=label_scaler.fit_transform(np.asarray(housing_labels).reshape(-1,1))[:,0]
housing_clean = housing_clean.drop("median_house_value", axis=1) # drop labels for training set
housing_cat = housing_clean[["ocean_proximity"]]
housing_num = housing_clean.drop("ocean_proximity", axis=1)

In [ ]:
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),#hay mas opciones aca
        ('std_scaler', StandardScaler()),
    ])

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

housing_prepared = full_pipeline.fit_transform(housing_clean)

Aca pasaron muchas cosas... Vamos paso por paso

El `num_pipeline` tiene dos pasos. Primero, el `SimpleImputer` se ocupa de rellenar los datos faltantes. Utiliza la mediana del feature faltante. Segundo, el `StandardScaler()` se ocupa de estandarizar los datos

In [ ]:
housing_num_transformed=num_pipeline.fit_transform(housing_num)

Veamos el imputer

In [ ]:
housing_num.info()

In [ ]:
len(housing_num_transformed[np.where(housing_num['total_bedrooms'].isnull()>0)])

In [ ]:
housing_num_transformed[np.where(housing_num['total_bedrooms'].isnull()>0),4]

Y el StandardScaler

In [ ]:
num_pipeline.named_steps['std_scaler'].mean_

In [ ]:
housing_prepared.shape

# Regresion


Jueguen ustedes un poco con las distintas opciones de modelo que damos a continuacion. Elijan cual les parece el mejor argumentandolo. El pseudo codigo de lo **minimo** que hay que hacer es el siguiente:

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

''' pseudo codigo '''

modelo = algoritmo()

modelo.fit(algo...)

y_pred_train = modelo.predict(algo...)

print(metrica(algo...))


Recuerden que se puede ver la documentacion con 

In [ ]:
LinearRegression?

Y, si pueden, vayan mas alla del pseudo-codigo porque hay un trampa...

# Mis soluciones (no lo vean antes de terminar con lo otro...)

Vamos a resolver el problema. Voy a probar varios algoritmos (sin justificarlos bien, ya los vamos a ver) y evaluar la performance.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import tree

## Underfitting con Regresion Lineal

Defino otro Pipeline solo para mostrar, no es estrictamente necesario.

$y = w_0 + \sum_{i=1}^{M}w_{i}\phi_{i}(\vec{x})$

$\phi_{i} = x_{i} $

$y = w_0 + \sum_{i=1}^{16}w_{i}x_{i}$

In [ ]:
full_pipeline_with_predictor_lr = Pipeline([
        ("preparation", full_pipeline),
        ("linear", LinearRegression())
    ])

scores_lr=cross_val_score(full_pipeline_with_predictor_lr, housing, housing_labels,scoring="neg_mean_squared_error", cv=10)#no lo aplico en housing_prepared, no deberia cambiar pero igual
cross_scores_lr = np.sqrt(-scores_lr)

print("Puntajes:", cross_scores_lr)
print("Media:", cross_scores_lr.mean())
print("Desviacion Estandar:", cross_scores_lr.std())

full_pipeline_with_predictor_lr.fit(housing, housing_labels)
predictions_lr=full_pipeline_with_predictor_lr.predict(housing)
print("Ejemplo: ", (round(predictions_lr[100]),housing_labels[100]))
print("MSE Total del conjunto de entrenamiento:", np.sqrt(mean_squared_error(predictions_lr,housing_labels)))

In [ ]:
x=np.linspace(min(housing_labels),max(housing_labels),3)
plt.scatter(housing_labels,predictions_lr)
plt.plot(x,x,color='red')
plt.xlabel('t')
plt.ylabel('y')

## Overfitting con Decision Tree

In [ ]:
tree_reg = DecisionTreeRegressor(random_state=42,max_depth=7)#,max_depth=5
tree_reg.fit(housing_prepared, housing_labels)


In [ ]:
x=np.linspace(min(housing_labels),max(housing_labels),3)
plt.scatter(housing_labels,tree_reg.predict(housing_prepared))
plt.plot(x,x,color='red')

In [ ]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

In [ ]:
tree_reg.get_n_leaves()

In [ ]:
scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

print("Puntajes:", tree_rmse_scores)
print("Media:", tree_rmse_scores.mean())
print("Desviacion Estandar:", tree_rmse_scores.std())

In [ ]:
tree.plot_tree(tree_reg) 
plt.show()

## Fitting con RandomForest

In [ ]:
param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    # then try 6 (2×3) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(housing_prepared, housing_labels)

In [ ]:
print("Best params:", grid_search.best_params_)
print("Best estimator:", grid_search.best_estimator_)


In [ ]:
cvres = grid_search.cv_results_
for mean_score, std, params in zip(cvres["mean_test_score"], cvres["std_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), np.sqrt(std),params)

In [ ]:
best_rf=grid_search.best_estimator_
print("Example: ", round(best_rf.predict(housing_prepared)[100]),housing_labels[100])
print("Train MSE: ",np.sqrt(mean_squared_error(best_rf.predict(housing_prepared),housing_labels)))

In [ ]:
x=np.linspace(min(housing_labels),max(housing_labels),3)
plt.scatter(housing_labels,best_rf.predict(housing_prepared))
plt.plot(x,x,color='red')

## Una Red Neuronal

In [ ]:
import tensorflow as tf
from tensorflow import keras
tf.random.set_seed(42)
keras.backend.clear_session()


In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(housing_num, housing_labels, random_state=42)
X_train_proc=num_pipeline.fit_transform(X_train)
X_valid_proc=num_pipeline.transform(X_valid)

In [ ]:
# print(np.asarray(y_train).reshape(-1,1).shape)

scaler = StandardScaler()
y_train_proc = scaler.fit_transform(np.asarray(y_train).reshape(-1,1))
y_valid_proc = scaler.transform(np.asarray(y_valid).reshape(-1,1))


In [ ]:
y_train_proc[:,0].shape

In [ ]:
input_shape = X_train_proc.shape[1:]
batch_size = 128
epochs = 50

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=input_shape),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])

In [ ]:

model.compile(loss="mean_squared_error", optimizer=keras.optimizers.SGD(lr=1e-3))

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model)

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True)
history = model.fit(X_train_proc, y_train_proc[:,0], epochs=epochs,
                    validation_data=(X_valid_proc, y_valid_proc[:,0]),
                    callbacks=[early_stopping_cb])

In [ ]:
pd.DataFrame(history.history).plot()
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

In [ ]:
mse_train = model.evaluate(X_train_proc, y_train_proc)

In [ ]:
x=np.linspace(min(y_train_proc[:,0]),max(y_train_proc[:,0]),3)
plt.scatter(y_train_proc[:,0],model.predict(X_train_proc))
plt.plot(x,x,color='red')

In [ ]:
np.sqrt(mean_squared_error(scaler.inverse_transform(model.predict(X_train_proc)),scaler.inverse_transform(y_train_proc)))

# Vamos al Test

En clase podemos hacerlo incompleto...

In [ ]:
housing_test_clean["rooms_per_household"] = housing_test_clean["total_rooms"]/housing_test_clean["households"]
housing_test_clean["bedrooms_per_room"] = housing_test_clean["total_bedrooms"]/housing_test_clean["total_rooms"]
housing_test_clean["population_per_household"]=housing_test_clean["population"]/housing_test_clean["households"]

In [ ]:
housing_test_labels = housing_test_clean["median_house_value"].copy()
housing_test_clean = housing_test_clean.drop("median_house_value", axis=1) # drop labels for training set
housing_test_cat = housing_test_clean[["ocean_proximity"]]
housing_test_num = housing_test_clean.drop("ocean_proximity", axis=1)

In [ ]:
housing_test_prepared = full_pipeline.transform(housing_test_clean)

Evaluo el mejor algoritmo: RandomForest

In [ ]:
print("Test MSE: ",np.sqrt(mean_squared_error(best_rf.predict(housing_test_prepared),housing_test_labels)))

# Algunos ejericicios (que pueden ser para dentro de unas clases...)





*   Esta bueno poder mostrar un grafico lindo. En particular, el mapa de latitud y longitud es bastante claro. Jueguen con los tres algoritmos que utilizamos pero ahora utilizando como features latitud y longitud. Para cada algoritmo dibuje el mapa y las regiones inferidas de precio utilizando plt.contourf. Por que sugerimos utilizar unicamente dos variables a la hora de entrenar en lugar de utilizar los algoritmos ya entrenados?
*   Reemplacen el GridSearchCV por el RandomizedSearchCV. No se preocupen, lo vamos a ver en detalle mas adelante.
*   Fijense si puede juntar el preprocesado de los datos y los distintos algoritmos en un solo Pipeline. Que hiperparametros tiene? Pueden implementarlos en GridSearchCV?


